lets import stuff.

In [5]:
import os
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import pandas as pd
from torchvision import models
import torch.nn as nn
from fastai.vision import *
import torch
from torch.autograd import Variable

define some constants

In [6]:
NUMBER = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
ALPHABET = ['a', 'b','c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

ALL_CHAR_SET = NUMBER + ALPHABET
ALL_CHAR_SET_LEN = len(ALL_CHAR_SET)
MAX_CAPTCHA = 5

one hot encoding of all characters

In [7]:
def encode(a):
    onehot = [0] * ALL_CHAR_SET_LEN
    idx = ALL_CHAR_SET.index(a)
    onehot[idx] +=1
    return onehot

In [8]:
import os

In [9]:
class Mydataset(Dataset):
    def __init__(self, path, is_train=True, transform=None):
        self.path = path
        if is_train: self.img = os.listdir(self.path)[:1000]
        else: self.img = os.listdir(self.path)[1001:]
        try: self.img.remove('3bnfnd.png')
        except: pass
        self.transform = transform
        
    def __getitem__(self, idx):
        img_path = self.img[idx]
        img = Image.open(self.path/img_path)
        img = img.convert('L')
        label = Path(self.path/img_path).name[:-4]
        label_oh = []
        for i in label:
            label_oh += encode(i)
        if self.transform is not None:
            img = self.transform(img)
        return img, np.array(label_oh), label
    
    def __len__(self):
        return len(self.img)

In [10]:
transform = transforms.Compose([
    transforms.Resize([224,244]),
    transforms.ToTensor()
])

In [11]:
from fastai.imports import Path

../input/captcha-version-2-images/null

../input/captcha-version-2-images/null


../input/captcha-version-2-images/samples/226md.png

../input/captcha-version-2-images/nul

../input/captcha-version-2-images/nulll


In [12]:
train_ds = Mydataset(Path('../input/captcha-version-2-images/samples/'), transform=transform)
test_ds = Mydataset(Path('../input/captcha-version-2-images/samples/'), False, transform)
train_dl = DataLoader(train_ds, batch_size=64, num_workers=0)
test_dl = DataLoader(train_ds, batch_size=1, num_workers=0)

In [13]:
model = models.resnet18(pretrained=False)

In [14]:
model.conv1 = nn.Conv2d(1,64,kernel_size=(7,7), stride=(2,2), padding=(3,3), bias=False)

In [15]:
model.fc = nn.Linear(in_features=512, out_features=ALL_CHAR_SET_LEN * MAX_CAPTCHA, bias = True)

Similarly, you can look at all the model parameters.

In [16]:
model.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('conv1',
               Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)),
              ('bn1',
               BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)),
              ('relu', ReLU(inplace=True)),
              ('maxpool',
               MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)),
              ('layer1',
               Sequential(
                 (0): BasicBlock(
                   (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runn

In [17]:
model.layer4

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(512, eps=1

In [18]:
model.cuda()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
loss_func = nn.MultiLabelSoftMarginLoss()
optm = torch.optim.Adam(model.parameters(), lr=0.001)

In [32]:
for epoch in range(20):
    try:
        for step, i in enumerate(train_dl):
            img, label_oh, label = i
#             print(label)
            img = Variable(img).cuda()
            label_oh = Variable(label_oh.float()).cuda()
            try:
                pred = model(img)
            except Exception as e:
                print(e)
            loss = loss_func(pred, label_oh)
            optm.zero_grad()
            loss.backward()
            optm.step()
            print('epoch:', epoch+1, 'step:', step+1, 'loss:', loss.item())
    except Exception as e:
        print(e)

epoch: 1 step: 1 loss: 0.06651271879673004
epoch: 1 step: 2 loss: 0.06531093269586563
epoch: 1 step: 3 loss: 0.0628591999411583
[Errno 21] Is a directory: '/kaggle/input/captcha-version-2-images/samples/samples'
epoch: 2 step: 1 loss: 0.060944393277168274
epoch: 2 step: 2 loss: 0.061414800584316254
epoch: 2 step: 3 loss: 0.056955333799123764
[Errno 21] Is a directory: '/kaggle/input/captcha-version-2-images/samples/samples'
epoch: 3 step: 1 loss: 0.05694485455751419
epoch: 3 step: 2 loss: 0.055977433919906616
epoch: 3 step: 3 loss: 0.05244304984807968
[Errno 21] Is a directory: '/kaggle/input/captcha-version-2-images/samples/samples'
epoch: 4 step: 1 loss: 0.05149337276816368
epoch: 4 step: 2 loss: 0.05101314187049866
epoch: 4 step: 3 loss: 0.047078829258680344
[Errno 21] Is a directory: '/kaggle/input/captcha-version-2-images/samples/samples'
epoch: 5 step: 1 loss: 0.04630549997091293
epoch: 5 step: 2 loss: 0.04567154496908188
epoch: 5 step: 3 loss: 0.04224460572004318
[Errno 21] Is a

In [33]:
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [36]:
import matplotlib.pyplot as plt

In [39]:
for step, (img, label_oh, label) in enumerate(test_dl):
    img = Variable(img).cuda()
    pred = model(img)
    c0 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[0:ALL_CHAR_SET_LEN])]
    c1 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN:ALL_CHAR_SET_LEN*2])]
    c2 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*2:ALL_CHAR_SET_LEN*3])]
    c3 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*3:ALL_CHAR_SET_LEN*4])]
    c4 = ALL_CHAR_SET[np.argmax(pred.squeeze().cpu().tolist()[ALL_CHAR_SET_LEN*4:ALL_CHAR_SET_LEN*5])]
    c = '%s%s%s%s%s' % (c0, c1, c2, c3, c4)

    print('label:', label[0], 'pred:', c)

torch.Size([1, 1, 224, 244])


RuntimeError: number of dims don't match in permute